In [34]:
import csv
from bs4 import BeautifulSoup
import re
import os
import pandas as pd
from lxml import etree
try:
    # Python 2
    from urllib2 import urlopen
except ImportError:
    from urllib.request import urlopen
import dateutil
from dateutil.parser import *
from datetime import datetime


In [3]:
csv_main = csv.DictReader(open("/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GithubRepository/Analysis/PYTHON/Scraper_v1/AgileScraper/AgileScraper/agile_sites_output.csv"), fieldnames=["id", 'url', 'domain', 'date1', 'date2', 'date3', 'heading'])

In [4]:
array = {}

In [5]:
for line in csv_main: 
    array[line["id"]] = line

In [85]:
agile_regex = re.compile(r'\bagile|\bagility', re.IGNORECASE | re.UNICODE)
agile_context_regex = re.compile(r'\s*([^\s]+?)\s+([^\s]+?)\s+([^\s]+?)\s+([^\s]+?)\s+agil.*?\s+([^\s]+)\s+([^\s]+?)\s+([^\s]+?)\s+([^\s]+?)\s+', re.IGNORECASE | re.UNICODE)
text_dir = "/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GithubRepository/Analysis/PYTHON/Analysis/DATA/TextFiles"

date5_xpath = "substring(substring-after(/html//script[@type='application/ld+json']/text(), 'datePublished'), 4, 24)"

for id, line in array.items():
    print("processing {}".format(id))
    id = line["id"]
    domain = line["domain"]
    path = '/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GithubRepository/Analysis/ALL_CRAWLS/Germany/Federal/{}/{}.html'.format(domain, id)
    try:  
        soup = BeautifulSoup(open(path), "html.parser")
    except UnicodeDecodeError:
        soup = BeautifulSoup(open(path, encoding='windows-1252'), "html.parser")    
    doctext = soup.get_text().replace("\n", " ")
    with open("{}/{}.txt".format(text_dir, id), "w") as textfile:
        textfile.write(doctext)
    
    date4_element = soup.select_one("span.date")
    date4 = ""
    if date4_element is not None:
        date4 = date4_element.get_text()
    
    #Getting date5

    htmlparser = etree.HTMLParser()    
    try:
        tree = etree.parse(open(path), htmlparser)
    except UnicodeDecodeError:
        tree = etree.parse(open(path, encoding='windows-1252'), htmlparser)
    date5 = tree.xpath(date5_xpath)

    # Getting date6

    date6_element = soup.select_one("h1#page-title + p")
    date6 = ""
    if date6_element is not None:
        date6 = date6_element.get_text()


    # Storing the oldest date as final date variable in python date format
    date_vars = ['date1', 'date2', 'date4', 'date5', 'date6']
    

    final_date = datetime(2050, 1, 1, 0, 0)

    for date_var in date_vars:
        try:
            if final_date == datetime(2050, 1, 1, 0, 0):
                final_date = parse(date_var)
            elif parse(date_var) < final_date:
                final_date = parse(date_var)
        except dateutil.parser._parser.ParserError:
            pass 



    agil_term = agile_regex.findall(doctext)
    agil_context = agile_context_regex.search(doctext)
    agil_context_pre = ""
    agil_context_post = ""
    if agil_context is not None:
        agil_context_pre = " ".join(agil_context.group(1,2,3,4))
        agil_context_post = " ".join(agil_context.group(5,6,7,8))
    if len(agil_term) == 0:
        line["agil_term"] = "" 
    else:
        line["agil_term"] = ",".join(agil_term)
    line["agil_context_pre"] = agil_context_pre
    line["agil_context_post"] = agil_context_post 
    line["date4"] = date4
    line["date5"] = date5
    line["date6"] = date6
    line["final_date"] = final_date
    line["country"] = "Britain"
    line["level"] = "general"
    line["text_file_loc"] = "{}/{}.txt".format(text_dir, id)

processing 74ef9f72-3612-3d2b-a14c-e080bb3567eb


AttributeError: type object 'datetime.datetime' has no attribute 'parse'

In [83]:
outputfile = open("/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GithubRepository/Analysis/PYTHON/Analysis/DATA/UK.csv", "w")
writer = csv.DictWriter(outputfile, fieldnames=["id", "country", "level", 'url', 'domain', 'date1', 'date2', 'date3', 'date4', 'date5', 'date6', 'final_date', 'heading', 'agil_term', 'agil_context_pre', 'agil_context_post', 'text_file_loc'])
writer.writeheader()
for id, line in array.items():
    writer.writerow(line)
outputfile.close()

In [84]:
#deleting double rows (if any) and rows where agil_term is empty hence not "agile" or "agility"

inputFileName = "/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GithubRepository/Analysis/PYTHON/Analysis/DATA/UK.csv"
outputFileName = "/Users/mxm/Google Drive/Masterstudium/Inhalte/Master Thesis/GithubRepository/Analysis/PYTHON/Analysis/DATA/UK_v1.csv"


with open(inputFileName, newline='') as inFile, open(outputFileName, 'w', newline='') as outFile:
    df = pd.read_csv(inFile)
    df.dropna(axis=0, how='any', thresh=None, subset=(['agil_term', 'agil_context_pre', 'agil_context_post']), inplace=True)
    df.drop_duplicates(inplace=True)
    df.to_csv(outFile)